# Capstone Project Final : Best location for opening an Indian restaurant in Manhattan




## Table of contents
* [1 - Introduction: Business Problem](#introduction)
* [2 - Data Description to solve the problem](#data)
* [3 - Methodology](#methodology)
* [4 - Analysis](#analysis)
* [5 - Results and Discussion](#results)
* [6 - Conclusion](#conclusion)

## 1 - Introduction: Business Problem <a name="introduction"></a>

The goal of the project is to open a Indian Restaurant in Manhattan, so the business problem, is to find the best place for opening the restaurant.
The target will be a investor who want to opening a restaurant, and to help him to find the best place base on Data Science algorithm (K-means Cluster).

The main obejective is to find the best neighborhood for opening based on these criterias :
* Attractivity of the area
* no competition (no other Indian restaurant nearby)
* great potential (good review, attractive, crowded area)
* Area with lot of restaurant







## 2 - Data Description to solve the problem <a name="data"></a>

For this Business problem, we will use the API FourSquare for :

* Get and analyse the venues of Paris
* Finding the neighborhoods, where you can find Lebanese restaurant
* Analyse each fo these neighborhood to find the best one, based on these criterias : Good revenues, Attractive area, good reviews etc...
* Find a similar neighborhood, but with no Indian restaurant opened yet.


    
    

### Restaurant analysis in Manhattan

We will import the useful livbraries and use FourQuare API for the analysis

In [1]:
# Import Libraries

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from bs4 import BeautifulSoup #Library for web scrapping
import urllib.request 

import json # library to handle JSON files


from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans



#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#!pip install folium
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


#### Downloading the data set of New York Neighborhood

In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)


In [4]:
neighborhoods_data = newyork_data['features']
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

#### Transform the data into a pandas dataframe

In [5]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

loop through the data and fill the dataframe one row at a time.

In [6]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [7]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


Display the shape of the data set

We will focus only on Manhattan sector

In [8]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [9]:
print('The dataframe of Manahatan has {} neighborhoods.'.format(manhattan_data.shape[0]
    )
)

The dataframe of Manahatan has 40 neighborhoods.


Geographical coordinates of Manhattan : 

In [10]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7900869, -73.9598295.


Now we will vizualize Manhatan and all it's neigborhood 

In [11]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

The next step is to use Foursquare API to explore and analyse the neigborhood, and to find where are the Indian restaurant

#### Foursquare Credentials and Version definition

In [12]:

CLIENT_ID = 'UQORBT4J0DMW2MWBCIFHZHJM3KJA1G0ZZOBAHNXVTNDOYWYO' # your Foursquare ID
CLIENT_SECRET = '5VBPSSBVFQFFMILW50QUEK2GHIBD05ZIFNA1YUW10TOK01N5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
LIMIT = 100 

Your credentails:
CLIENT_ID: UQORBT4J0DMW2MWBCIFHZHJM3KJA1G0ZZOBAHNXVTNDOYWYO
CLIENT_SECRET:5VBPSSBVFQFFMILW50QUEK2GHIBD05ZIFNA1YUW10TOK01N5


## 3 - Methodology <a name="methodology"></a>


In this project we will find the best neighborhoods for opening an indian Restaurant in Manhattan area in Ney York City.

So the objective is to find an area with many Indian Restaurant, then to find a similar area but with no or few indian restaurant, for avoiding competitions.

Here are the steps :

* The first step will be to analyse the venues for each enighborhood of Manahattan using the Foursquare API.

* The second step is to find for each Neighborhood the top ten Venues, then using K-means to cluser them.

* The third step is to define which cluster containing the more Indian restaurants.

* And finally to find which neighborhoods  corresponding to that cluster, but with few Indian restaurant( 0 or 1) for avoiding competition, and visualize it using a map.





#### Using Foursquare we will serch for Indian Restaurant by neighborhood


In [13]:
# type your answer here
##<!-- The correct answer is:
LIMIT = 1000 # limit of number of venues returned by Foursquare API
##-->

##<!--
radius = 500 # define radius

## 4 - Analysis <a name="analysis"></a>


#### Define a function to get  the venues  in Manhattan¶

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
## Get the venues for Manhatan area :

manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )


Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [16]:
LIMIT

1000

In [17]:

#Display the information
print(manhattan_venues.shape)
manhattan_venues.head()

(3331, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop


Here's are the number of venues returned for each neighborhood: 

In [18]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,100,100,100,100,100,100
Carnegie Hill,100,100,100,100,100,100
Central Harlem,47,47,47,47,47,47
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,100,100,100,100,100,100
Clinton,100,100,100,100,100,100
East Harlem,44,44,44,44,44,44
East Village,100,100,100,100,100,100


Now we will verify if we have Indian Restaurant In Manhattan adn display the Neighborhood:

In [19]:
df_Indian_rest = manhattan_venues[manhattan_venues['Venue Category'].str.contains('Indian Restaurant')]
df_Indian_rest

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
149,Washington Heights,40.851903,-73.936900,Kismat Indian Restaurant,40.855222,-73.936967,Indian Restaurant
297,Hamilton Heights,40.823604,-73.949688,Clove Indian Restaurant & Bar,40.821280,-73.950620,Indian Restaurant
311,Hamilton Heights,40.823604,-73.949688,Mumbai Masala,40.826866,-73.946486,Indian Restaurant
343,Manhattanville,40.816934,-73.957385,Chapati House - NYC,40.814572,-73.959154,Indian Restaurant
836,Upper West Side,40.787658,-73.977059,Alachi Masala,40.786111,-73.976270,Indian Restaurant
860,Upper West Side,40.787658,-73.977059,Ashoka,40.784842,-73.973253,Indian Restaurant
869,Upper West Side,40.787658,-73.977059,Swagat,40.783573,-73.978030,Indian Restaurant
896,Upper West Side,40.787658,-73.977059,Saravanaa Bhavan,40.783578,-73.977860,South Indian Restaurant
1177,Midtown,40.754691,-73.981669,The Kati Roll Company,40.752656,-73.984778,Indian Restaurant
1191,Midtown,40.754691,-73.981669,indikitch,40.757819,-73.982007,Indian Restaurant


Get the the number of Unique Venues

In [20]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 334 uniques categories.


### Data Preparation for clustering the Neighborhood of Manhattan

In [21]:
# we will trnaform the data frame in order to have the Venue Category in column
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Gym,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shoppin

In [22]:
manhattan_onehot.shape

(3331, 335)

##### Now we will group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [23]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Gym,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shoppin

In [24]:
manhattan_grouped.shape

(40, 335)

Select the neigborhood with Indian restaurant

In [38]:

df_Indian_rest = manhattan_grouped[manhattan_grouped['Indian Restaurant'] > 0]
df_Indian_rest

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Gym,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shoppin

##### Let's view for each neighborhood the top 10 Venues 

In [26]:
#Function  to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

DataFrame creation : the top 10 venues for each neighborhood.

In [27]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Coffee Shop,Hotel,Gym,Memorial Site,Wine Shop,Clothing Store,Italian Restaurant,Department Store,Women's Store
1,Carnegie Hill,Coffee Shop,Pizza Place,Café,Yoga Studio,Bookstore,Cosmetics Shop,French Restaurant,Bar,Japanese Restaurant,Spa
2,Central Harlem,African Restaurant,Art Gallery,Seafood Restaurant,American Restaurant,Gym / Fitness Center,French Restaurant,Cosmetics Shop,Chinese Restaurant,Public Art,Grocery Store
3,Chelsea,Coffee Shop,Italian Restaurant,Ice Cream Shop,Nightclub,Bakery,Seafood Restaurant,American Restaurant,Theater,Art Gallery,Hotel
4,Chinatown,Chinese Restaurant,Cocktail Bar,American Restaurant,Dumpling Restaurant,Spa,Ice Cream Shop,Bakery,Vietnamese Restaurant,Bubble Tea Shop,Dim Sum Restaurant


### Cluster Neighborhoods


We will use k-means to cluster the neighborhood into 5 clusters.


In [28]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 2, 2, 1, 2, 2, 0, 1, 1], dtype=int32)

Creation of a new data frame to have for each Neighborhood : The Cluster, and the 10 top Venues 

In [29]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,1,Coffee Shop,Discount Store,Sandwich Place,Yoga Studio,Tennis Stadium,Supplement Shop,Steakhouse,Spa,Seafood Restaurant,Clothing Store
1,Manhattan,Chinatown,40.715618,-73.994279,1,Chinese Restaurant,Cocktail Bar,American Restaurant,Dumpling Restaurant,Spa,Ice Cream Shop,Bakery,Vietnamese Restaurant,Bubble Tea Shop,Dim Sum Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,0,Café,Mobile Phone Shop,Bakery,Spanish Restaurant,Deli / Bodega,Mexican Restaurant,Sandwich Place,New American Restaurant,Park,Supplement Shop
3,Manhattan,Inwood,40.867684,-73.921210,0,Mexican Restaurant,Café,Lounge,Bakery,Pizza Place,Park,Frozen Yogurt Shop,Chinese Restaurant,American Restaurant,Wine Bar
4,Manhattan,Hamilton Heights,40.823604,-73.949688,0,Deli / Bodega,Café,Mexican Restaurant,Pizza Place,Chinese Restaurant,Coffee Shop,Sushi Restaurant,Caribbean Restaurant,Bank,Bakery


### Visualize the clusters on the maps


In [30]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Now let's see for each Neighborhood the number of Indian Restaurant

In [31]:
df_GroupByVenues =  manhattan_venues.loc[:,['Neighborhood','Neighborhood Latitude','Neighborhood Longitude','Venue Category']]

df_Indian_rest2 = manhattan_venues[manhattan_venues['Venue Category'].str.contains('Indian Restaurant')]
df_Indian_rest2
df_Indian_rest2_Count= df_Indian_rest2.groupby(['Neighborhood']).count().reset_index()
df_Indian_rest2_Count

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Carnegie Hill,2,2,2,2,2,2
1,Chelsea,1,1,1,1,1,1
2,Civic Center,1,1,1,1,1,1
3,Financial District,1,1,1,1,1,1
4,Flatiron,1,1,1,1,1,1
5,Gramercy,2,2,2,2,2,2
6,Greenwich Village,3,3,3,3,3,3
7,Hamilton Heights,2,2,2,2,2,2
8,Manhattan Valley,3,3,3,3,3,3
9,Manhattanville,1,1,1,1,1,1


Let' see the cluster apply for each of the neighborhood with indian Restaurant : 

In [32]:
# We will merge the two dataframes : 

Manhatan_indian_restaurant = pd.merge(df_Indian_rest2_Count, manhattan_merged)
Manhatan_indian_restaurant.sort_values(by=['Venue Category'],ascending=False)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,Upper West Side,4,4,4,4,4,4,Manhattan,40.787658,-73.977059,1,Italian Restaurant,Wine Bar,Bar,Vegetarian / Vegan Restaurant,Mediterranean Restaurant,Bakery,Indian Restaurant,Coffee Shop,Yoga Studio,Pub
13,Sutton Place,4,4,4,4,4,4,Manhattan,40.760280,-73.963556,1,Gym / Fitness Center,Italian Restaurant,Furniture / Home Store,Indian Restaurant,Juice Bar,Gym,Dessert Shop,Pizza Place,American Restaurant,Latin American Restaurant
15,Turtle Bay,3,3,3,3,3,3,Manhattan,40.752042,-73.967708,1,Italian Restaurant,Coffee Shop,Sushi Restaurant,Steakhouse,Ramen Restaurant,Wine Bar,Park,Hotel,French Restaurant,Indian Restaurant
6,Greenwich Village,3,3,3,3,3,3,Manhattan,40.726933,-73.999914,2,Italian Restaurant,Clothing Store,Sushi Restaurant,French Restaurant,Café,Seafood Restaurant,Chinese Restaurant,Indian Restaurant,Ice Cream Shop,Dessert Shop
8,Manhattan Valley,3,3,3,3,3,3,Manhattan,40.797307,-73.964286,0,Indian Restaurant,Coffee Shop,Pizza Place,Yoga Studio,Mexican Restaurant,Café,Bar,Thai Restaurant,Deli / Bodega,Szechuan Restaurant
0,Carnegie Hill,2,2,2,2,2,2,Manhattan,40.782683,-73.953256,1,Coffee Shop,Pizza Place,Café,Yoga Studio,Bookstore,Cosmetics Shop,French Restaurant,Bar,Japanese Restaurant,Spa
5,Gramercy,2,2,2,2,2,2,Manhattan,40.737210,-73.981376,1,Bar,Cocktail Bar,American Restaurant,Italian Restaurant,Bagel Shop,Pizza Place,Thai Restaurant,Mexican Restaurant,Hotel,Grocery Store
7,Hamilton Heights,2,2,2,2,2,2,Manhattan,40.823604,-73.949688,0,Deli / Bodega,Café,Mexican Restaurant,Pizza Place,Chinese Restaurant,Coffee Shop,Sushi Restaurant,Caribbean Restaurant,Bank,Bakery
10,Midtown,2,2,2,2,2,2,Manhattan,40.754691,-73.981669,1,Hotel,Coffee Shop,Cocktail Bar,Clothing Store,Theater,Steakhouse,Spa,American Restaurant,Sporting Goods Shop,Japanese Restaurant
12,Noho,1,1,1,1,1,1,Manhattan,40.723259,-73.988434,2,Italian Restaurant,Cocktail Bar,French Restaurant,Mexican Restaurant,Boutique,Sushi Restaurant,Sandwich Place,Rock Club,Coffee Shop,Pizza Place


#### Let's see on which cluster we have the maximum number of indian restaurant

In [33]:
df_cluster_By_Indian_Restaurant = Manhatan_indian_restaurant.groupby(['Cluster Labels']).count().reset_index()
df_cluster_By_Indian_Restaurant

,Cluster Labels,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Borough,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
1,1,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7
2,2,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7
3,4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


The cluster with the maximum number of indian restaurant are 1 or 2 : (7 indian restaurants for both)

##### Let's focus on these 2 clusters only and find Neighborhoods corresponding to these clusters , but with few indian restaurants:


In [34]:
array= [1,2]
df_location_new_ind_restaurant= Manhatan_indian_restaurant[Manhatan_indian_restaurant['Cluster Labels'].isin(array) ]
df_location_new_ind_restaurant.sort_values(by=['Venue Category'],ascending=True)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chelsea,1,1,1,1,1,1,Manhattan,40.744035,-74.003116,2,Coffee Shop,Italian Restaurant,Ice Cream Shop,Nightclub,Bakery,Seafood Restaurant,American Restaurant,Theater,Art Gallery,Hotel
2,Civic Center,1,1,1,1,1,1,Manhattan,40.715229,-74.005415,2,Gym / Fitness Center,French Restaurant,Hotel,Sandwich Place,Coffee Shop,Italian Restaurant,Bakery,Yoga Studio,Spa,Cocktail Bar
3,Financial District,1,1,1,1,1,1,Manhattan,40.707107,-74.010665,1,Coffee Shop,Steakhouse,Wine Shop,American Restaurant,Hotel,Gym,Cocktail Bar,Juice Bar,Pizza Place,Bar
4,Flatiron,1,1,1,1,1,1,Manhattan,40.739673,-73.990947,2,Gym,Yoga Studio,American Restaurant,Japanese Restaurant,Gym / Fitness Center,Clothing Store,Cosmetics Shop,Dessert Shop,Cycle Studio,Spa
12,Noho,1,1,1,1,1,1,Manhattan,40.723259,-73.988434,2,Italian Restaurant,Cocktail Bar,French Restaurant,Mexican Restaurant,Boutique,Sushi Restaurant,Sandwich Place,Rock Club,Coffee Shop,Pizza Place
14,Tribeca,1,1,1,1,1,1,Manhattan,40.721522,-74.010683,2,Italian Restaurant,Spa,Café,Park,American Restaurant,Boutique,Greek Restaurant,Gym,Wine Bar,Wine Shop
18,West Village,1,1,1,1,1,1,Manhattan,40.734434,-74.006180,2,Italian Restaurant,New American Restaurant,Cosmetics Shop,American Restaurant,Jazz Club,Park,Wine Bar,Bakery,Cocktail Bar,Coffee Shop
0,Carnegie Hill,2,2,2,2,2,2,Manhattan,40.782683,-73.953256,1,Coffee Shop,Pizza Place,Café,Yoga Studio,Bookstore,Cosmetics Shop,French Restaurant,Bar,Japanese Restaurant,Spa
5,Gramercy,2,2,2,2,2,2,Manhattan,40.737210,-73.981376,1,Bar,Cocktail Bar,American Restaurant,Italian Restaurant,Bagel Shop,Pizza Place,Thai Restaurant,Mexican Restaurant,Hotel,Grocery Store
10,Midtown,2,2,2,2,2,2,Manhattan,40.754691,-73.981669,1,Hotel,Coffee Shop,Cocktail Bar,Clothing Store,Theater,Steakhouse,Spa,American Restaurant,Sporting Goods Shop,Japanese Restaurant


Let's only select the neighborhhood with Only 1 Indian restaurant : 

In [35]:
df_final_location =df_location_new_ind_restaurant[df_location_new_ind_restaurant['Venue Category'] ==1 ]
df_final_location 

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chelsea,1,1,1,1,1,1,Manhattan,40.744035,-74.003116,2,Coffee Shop,Italian Restaurant,Ice Cream Shop,Nightclub,Bakery,Seafood Restaurant,American Restaurant,Theater,Art Gallery,Hotel
2,Civic Center,1,1,1,1,1,1,Manhattan,40.715229,-74.005415,2,Gym / Fitness Center,French Restaurant,Hotel,Sandwich Place,Coffee Shop,Italian Restaurant,Bakery,Yoga Studio,Spa,Cocktail Bar
3,Financial District,1,1,1,1,1,1,Manhattan,40.707107,-74.010665,1,Coffee Shop,Steakhouse,Wine Shop,American Restaurant,Hotel,Gym,Cocktail Bar,Juice Bar,Pizza Place,Bar
4,Flatiron,1,1,1,1,1,1,Manhattan,40.739673,-73.990947,2,Gym,Yoga Studio,American Restaurant,Japanese Restaurant,Gym / Fitness Center,Clothing Store,Cosmetics Shop,Dessert Shop,Cycle Studio,Spa
12,Noho,1,1,1,1,1,1,Manhattan,40.723259,-73.988434,2,Italian Restaurant,Cocktail Bar,French Restaurant,Mexican Restaurant,Boutique,Sushi Restaurant,Sandwich Place,Rock Club,Coffee Shop,Pizza Place
14,Tribeca,1,1,1,1,1,1,Manhattan,40.721522,-74.010683,2,Italian Restaurant,Spa,Café,Park,American Restaurant,Boutique,Greek Restaurant,Gym,Wine Bar,Wine Shop
18,West Village,1,1,1,1,1,1,Manhattan,40.734434,-74.006180,2,Italian Restaurant,New American Restaurant,Cosmetics Shop,American Restaurant,Jazz Club,Park,Wine Bar,Bakery,Cocktail Bar,Coffee Shop


#### Let's vizualise these selected neighborhood on the maps :


In [36]:
# create map
map_clusters_final = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_final_location['Latitude'], df_final_location['Longitude'], df_final_location['Neighborhood'], df_final_location['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters_final)
       
map_clusters_final

## 5 - Results and Discussion <a name="results"></a>

So we have foud 7 potential neighborhoods for opening an indian restaurants in Manahattan.
Their cluster are similar to the one where there are many Indian restaurants.And they have great potential because only 1 indian restaurant is opened in each of these neighborhood.

We can pursue a deeper analysis to find and select which of these neighborhood are the more attractives by using foursquare to get more informations about them such as : Review of the areas, attractivity( tourism) etc...
    


## 6 - Conclusion <a name="conclusion"></a>


In [41]:
The goal of the project was to find the best place for opening an indian  restaurant in Manhatan by using Data Science algorithm (K-means Cluster).

Finally we have use K-Means algorithm and foursqaure API to find the best clusters (1 or 2) containing a large number of indian restaurant.
and found 7 Neighborhood similar as thes clusters, but with just on Indian Restaurant, for avoiding competition.

So these areas has a great potential because there are similiar to areas with a large number of indian restaurant, the top venues a

find 7 potential Neighborhood
The main obejective is to find the best neighborhood for opening based on these criterias :
* Attractivity of the area
* no competition (no other Indian restaurant nearby)
* great potential (good review, attractive, crowded area)
* Area with lot of restaurant



SyntaxError: invalid syntax (<ipython-input-41-35b791ed7a83>, line 1)